In [95]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait as wait
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC

import time
#from webdriver_manager.chrome import ChromeDriverManager 

#s = Service("chromedriver.exe")
#PATH = "chromedriver.exe"
#driver = webdriver.Chrome(service=s)
driver = webdriver.Chrome()

driver.get("https://www.amazon.com")

In [96]:
keyword = "headset"
# assign any keyword for searching
search_box = driver.find_element(By.ID, 'twotabsearchtextbox')
# type the keyword in searchbox
search_box.send_keys(keyword)
# create WebElement for a search button
search_button = driver.find_element(By.ID, 'nav-search-submit-button')
# click search_button
search_button.click()
# wait for the page to download
driver.implicitly_wait(5)

In [128]:
# Arrays for keeping data
product_name = []
product_asin = []
product_price = []
product_ratings = []
product_ratings_num = []
product_link = []

In [129]:
while True:
    # Find all items on the page
    #items = driver.find_elements(By.XPATH, '//div[@data-asin]')
    
    items = wait(driver, 30).until(EC.presence_of_all_elements_located((By.XPATH, '//div[contains(@class, "s-result-item s-asin")]')))
    
    for item in items:
        # find name
        name = item.find_element(By.XPATH, './/span[@class="a-size-medium a-color-base a-text-normal"]')
        product_name.append(name.text)

        # find ASIN number 
        data_asin = item.get_attribute("data-asin")
        product_asin.append(data_asin)

        # find price
        whole_price = item.find_elements(By.XPATH, './/span[@class="a-price-whole"]')
        fraction_price = item.find_elements(By.XPATH, './/span[@class="a-price-fraction"]')

        if whole_price != [] and fraction_price != []:
            price = '.'.join([whole_price[0].text, fraction_price[0].text])
        else:
            price = 0
        product_price.append(price)

        # find ratings box
        ratings_box = item.find_elements(By.XPATH, './/div[@class="a-row a-size-small"]/span')

        # find ratings and ratings_num
        if ratings_box != []:
            ratings = ratings_box[0].get_attribute('aria-label')
            ratings_num = ratings_box[1].get_attribute('aria-label')
        else:
            ratings, ratings_num = 0, 0

        product_ratings.append(ratings)
        product_ratings_num.append(str(ratings_num))
    
    next_button = driver.find_elements(By.XPATH, '//a[@class="s-pagination-item s-pagination-next s-pagination-button s-pagination-separator"]')
    if len(next_button) == 0:
        break
    else:
        # Click the "Next" button to go to the next page
        next_button[0].click()
        time.sleep(10)  

In [144]:
import pandas as pd


product_price = [str(p).replace('\uff0c', ',') for p in product_price]
rows = zip(product_name, product_asin, product_price, product_ratings, product_ratings_num)

df = pd.DataFrame(rows, columns=['Name', 'Asin', 'Price', 'Rating', 'Rating_Num'])

df.to_excel("product_list.xlsx", index=False)


### Cleaning Data

In [145]:
df

,Name,Asin,Price,Rating,Rating_Num
0,HyperX - Cloud Core Wired DTS Headphone:X Gami...,B08HQW9SGP,29.99,4.5 out of 5 stars,"120,267"
1,"Logitech H390 Wired Headset for PC/Laptop, Ste...",B09XQ5WSF5,24.99,4.4 out of 5 stars,"49,368"
2,BENGOO G9000 Stereo Gaming Headset for PS4 PC ...,B01H6GUCCQ,25.99,4.3 out of 5 stars,"98,456"
3,SENZER SG500 Surround Sound Pro Gaming Headset...,B08FX35S7K,29.99,4.3 out of 5 stars,"8,982"
4,"HyperX Cloud Stinger – Gaming Headset, Lightwe...",B0BHV2JT81,49.99,4.4 out of 5 stars,"22,383"
...,...,...,...,...,...
421,New bee Wireless Headset Bluetooth Headset wit...,B0BRRCJM6R,39.99,4.3 out of 5 stars,91
422,USB Headset for Zoom Online Meeting Video Conf...,B08L3FRW89,25.99,4.2 out of 5 stars,100
423,"KUNSLUCK PS VR2 Accessories Kit, Lens Protecto...",B0BX9CTCN8,13.99,3.5 out of 5 stars,2
424,Gaming Headset with 7.1 Surround Sound and Noi...,B0B82SLGB3,19.99,0,0


In [150]:
print(df['Name'].dtype)
print(df['Asin'].dtype)
print(df['Price'].dtype)
print(df['Rating'].dtype)
print(df['Rating_Num'].dtype)

object
object
object
object
object


In [163]:
df['Price'] = df['Price'].astype(float)
df['Name'] = df['Name'].astype(str)
df['Asin'] = df['Asin'].astype(str)
df['Name'] = df['Name'].astype(str)
df['Name'] = df['Name'].astype(str)


In [164]:
print(df['Price'].dtype)


float64
